In [1]:
import torch
import torchvision
import detectron2,cv2
from detectron2.utils.logger import setup_logger
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
import time
setup_logger()
print(torch.__version__)

2.2.2


In [2]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.DEVICE = "cpu" # cpu or cuda
predictor = DefaultPredictor(cfg)

[04/22 16:36:06 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...


In [3]:

def find_person_indicies(scores):
    return [i for i, s in enumerate(scores) if s > 0.9]


In [4]:

def filter_persons(outputs):
    persons = {}
    p_indicies = find_person_indicies(outputs["instances"].scores)
    for x in p_indicies:
        desired_kp = outputs["instances"].pred_keypoints[x][:].to("cpu")
        persons[x] = desired_kp
    return (persons, p_indicies)

In [5]:
WHITE_COLOR = (255, 255, 255)
GREEN_COLOR = (0, 255, 0)

In [25]:
def draw_line(image, p1, p2, color):
    print(p1, p2)
    cv2.line(image, p1, p2, color, thickness=2, lineType=cv2.LINE_AA)



In [37]:
def draw_keypoints(person, img):
    l_eye = person[1]
    r_eye = person[2]
    l_ear = person[3]
    r_ear = person[4]
    nose = person[0]
    l_shoulder = person[5]
    r_shoulder = person[6]
    l_elbow = person[7]
    r_elbow = person[8]
    l_wrist = person[9]
    r_wrist = person[10]
    l_hip = person[11]
    r_hip = person[12]
    l_knee = person[13]
    r_knee = person[14]
    l_ankle = person[15]
    r_ankle = person[16]
    draw_line(img, (int(round(l_shoulder[0].item())), int(round(l_shoulder[1].item()))),
              (int(round(l_elbow[0].item())), int(round(l_elbow[1].item()))), GREEN_COLOR)
    

In [8]:
SKIP_FRAME_COUNT = 0

def analyse_video(pose_detector, video_path):
    # open the video
    cap = cv2.VideoCapture(video_path)
    # width of image frame
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # height of image frame
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # frames per second of the input video
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # total number of frames in the video
    tot_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
     # video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    file_name = video_path.split(os.path.sep)[-1]
    vid_writer = cv2.VideoWriter('res_{}'.format(
        file_name), fourcc, 30, (width, height))
    # counter
    counter = 0
    # buffer to keep the output of detectron2 pose estimation
    buffer_window = []
    # start time
    start = time.time()
    label = None
    # iterate through the video
    while True:
        # read the frame
        ret, frame = cap.read()
        # return if end of the video
        if ret == False:
            break
        # make a copy of the frame
        img = frame.copy()
        if(counter % (SKIP_FRAME_COUNT+1) == 0):
            # predict pose estimation on the frame
            outputs = pose_detector(frame)
            # filter the outputs with a good confidence score
            persons, pIndicies = filter_persons(outputs)
            if len(persons) >= 1:
                # pick only pose estimation results of the first person.
                # actually, we expect only one person to be present in the video.
                p = persons[0]
                draw_keypoints(p, img)
                # input feature array for lstm
                features = []
                # add pose estimate results to the feature array
                for i, row in enumerate(p):
                    features.append(row[0].item())
                    features.append(row[1].item())

        # increment counter
        counter += 1
         # write the frame into the result video
        vid_writer.write(img)
        # compute the completion percentage
        percentage = int(counter*100/tot_frames)
        # return the completion percentage
        #yield "data:" + str(percentage) + "\n\n"
        yield ','.join(str(x) for x in features)
    analyze_done = time.time()
    print("Video processing finished in ", analyze_done - start)

In [7]:
r = []
for root, dirs, files in os.walk('../data/videos'):
    for name in files:
        filepath = root + os.sep + name
        if filepath.endswith(".mp4"):
            if os.path.isfile(filepath.replace('.mp4','.csv')):
                continue
            f = open('../Detectron/Train/'+filepath.replace('.mp4','.csv'), 'a')
            try:
                for i in analyse_video(predictor, filepath):
                    f.write(i+'\n')
            except:
                print("Error processing ", filepath)
            f.close()

/Users/kanantharaman/anaconda3/envs/my_env/lib/python3.9/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711403209812/work/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Video processing finished in  436.63010001182556
Video processing finished in  369.00505685806274
Video processing finished in  345.3452899456024
Video processing finished in  346.46355724334717
Video processing finished in  344.9829921722412
Video processing finished in  341.167072057724
Video processing finished in  367.5799219608307
Video processing finished in  360.1744577884674
Video processing finished in  161.35592985153198
Video processing finished in  163.52269387245178
Video processing finished in  470.40084195137024
Video processing finished in  142.19026517868042
Video processing finished in  242.9505259990692
Video processing finished in  431.12960171699524
Video processing finished in  237.2165491580963
Video processing finished in  2228.9235661029816
Video processing finished in  338.89612579345703
Video processing finished in  140.32995796203613
Video processing finished in  381.58306312561035
Video processing finished in  244.29811191558838
Video processing finished in

: 